# Design the paired states with sequence symmetry, sampling backbones for MPNN while maintaining sequence realism

### Imports

In [1]:
%load_ext lab_black
# Python standard library
from glob import glob
import os
import socket
import sys

# 3rd party library imports
import dask
import matplotlib.pyplot as plt
import pandas as pd
import pyrosetta
import numpy as np
import scipy
import seaborn as sns
from tqdm.auto import tqdm  # jupyter compatible progress bar

tqdm.pandas()  # link tqdm to pandas
# Notebook magic
# save plots in the notebook
%matplotlib inline
# reloads modules automatically before executing cells
%load_ext autoreload
%autoreload 2
print(f"running in directory: {os.getcwd()}")  # where are we?
print(f"running on node: {socket.gethostname()}")  # what node are we on?

running in directory: /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties
running on node: dig41


### Set working directory to the root of the crispy_shifty repo
TODO set to projects dir

In [2]:
os.chdir("/home/pleung/projects/crispy_shifty")
# os.chdir("/projects/crispy_shifty")

### Design the paired states
TODO

In [3]:
# from crispy_shifty.utils.io import gen_array_tasks

# simulation_name = "05_design_paired_states"
# design_list_file = os.path.join(
#     os.getcwd(),
#     "projects/crispy_shifties/04_pair_bound_states/paired_states.list",
# )
# output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

# options = " ".join(
#     [
#         "out:level 200",
#         "corrections::beta_nov16 true",
# #         "indexed_structure_store:fragment_store /net/databases/VALL_clustered/connect_chains/ss_grouped_vall_helix_shortLoop.h5",
#     ]
# )

# gen_array_tasks(
#     distribute_func="crispy_shifty.protocols.msd.design_paired_state",
#     design_list_file=design_list_file,
#     output_path=output_path,
#     queue="short",
#     memory="4G",
#     nstruct=1,
#     nstruct_per_task=1,
#     options=options,
#     simulation_name=simulation_name,
# )

In [4]:
# !sbatch -a 1-$(cat /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/02_mpnn_bound_states/tasks.cmds | wc -l) /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/02_mpnn_bound_states/run.sh

### Collect scorefiles of designed paired states and concatenate
TODO change to projects dir

In [5]:
# sys.path.insert(0, "~/projects/crispy_shifty")  # TODO
# from crispy_shifty.utils.io import collect_score_file

# simulation_name = "05_design_paired_states"
# output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

# if not os.path.exists(os.path.join(output_path, "scores.json")):
#     collect_score_file(output_path, "scores")

### Load resulting concatenated scorefile
TODO change to projects dir

In [6]:
# sys.path.insert(0, "~/projects/crispy_shifty")  # TODO
# from crispy_shifty.utils.io import parse_scorefile_linear

# output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

# scores_df = parse_scorefile_linear(os.path.join(output_path, "scores.json"))
# scores_df = scores_df.convert_dtypes()

### Setup for plotting

In [7]:
# sns.set(
#     context="talk",
#     font_scale=1,  # make the font larger; default is pretty small
#     style="ticks",  # make the background white with black lines
#     palette="colorblind",  # a color palette that is colorblind friendly!
# )

### Data exploration
Gonna remove the Rosetta sfxn scoreterms for now

In [8]:
# from crispy_shifty.protocols.design import beta_nov16_terms

# scores_df = scores_df[
#     [term for term in scores_df.columns if term not in beta_nov16_terms]
# ]
# print(len(scores_df))

In [9]:
# print(list(scores_df.columns))

In [10]:
# from crispy_shifty.plotting.utils import histplot_df, pairplot_df

# to_plot = [
#     "bb_clash_delta_x",
#     "score_per_res_x",
#     "wnm_all_x",
# ]
# print(to_plot)

### Filter extreme outliers and changes some dtypes

In [11]:
# scores_df["score_per_res_x"] = scores_df["score_per_res_x"].astype(float)

# query = "bb_clash_delta_x < 500 and score_per_res_x < 0 and wnm_all_x < 5"
# sample_df = scores_df.query(query)

### Plot loop scores

In [12]:
# cols = [
#     "bb_clash_delta_x",
#     "score_per_res_x",
#     "wnm_all_x",
# ]
# the_fig = histplot_df(
#     df=sample_df,
#     cols=cols,
#     bins=10,
#     hue="scaffold_type",
# )
# plt.savefig(os.path.join(output_path, "loop_scores.png"))

### Filter out obviously bad decoys

In [13]:
# query = "bb_clash_delta_x < 200 and score_per_res_x < -2 and wnm_all_x < 1"
# filtered = sample_df.query(query)
# len(filtered)

### Save a list of outputs

In [14]:
# simulation_name = "05_design_paired_states"
# output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

# with open(os.path.join(output_path, "designed_paired_states.list"), "w") as f:
#     for path in tqdm(filtered.index):
#         print(path, file=f)

### Prototyping blocks

test `design_paired_state`

In [15]:
%%time 
import pyrosetta

pyrosetta.init(
    "-corrections::beta_nov16 \
    -indexed_structure_store:fragment_store /net/databases/VALL_clustered/connect_chains/ss_grouped_vall_helix_shortLoop.h5"
)

sys.path.insert(0, "~/projects/crispy_shifty/") # TODO projects
from crispy_shifty.protocols.msd import two_state_design_paired_state

t = two_state_design_paired_state(
        None,
        **{
            'pdb_path': '/mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/04_pair_bound_states/decoys/0001/04_pair_bound_states_331c8b82841d4de280ff3db199bb973f.pdb.bz2'
        }
)
for i, tppose in enumerate(t):
    tppose.pose.dump_pdb(f"{i}.pdb")

PyRosetta-4 2021 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python38.Release 2021.50+release.4ff291ed8257ec2d0cd4d96ae4289e1d39ce1007 2021-12-16T00:25:15] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python38.Release r306 2021.50+release.4ff291e 4ff291ed8257ec2d0cd4d96ae4289e1d39ce1007 http://www.pyrosetta.org 2021-12-16T00:25:15
core.init: command: PyRosetta -corrections::beta_nov16 -indexed_structure_store:fragment_store /net/databases/VALL_clustered/connect_chains/ss_grouped_vall_helix_shortLoop.h5 -database /projects/crispy_shifty/envs/crispy/lib/python3.8/site-packages/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-631392715 seed_offset=0 real_seed=-6313927

/projects/crispy_shifty/envs/crispy/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBEval.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/DonStrength.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/AccStrength.csv
core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 983 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 1.2887 seconds.
basic.io.database: Database file opened: scoring/score_functions/rama/fd_beta_nov2016/all.ramaProb
basic.io.database: Database file opened: scoring/score_functions/rama/fd_beta_nov2016/prepro.ramaProb
basic.io.database: Database file opened

RuntimeError: 

File: /home/benchmark/rosetta/source/src/protocols/rosetta_scripts/RosettaScriptsParser.cc:1192
Input rosetta scripts XML file "XmlString (not real file)" failed to validate against the rosetta scripts schema. Use the option -parser::output_schema <output filename> to output the schema to a file to see all valid options.
Your XML has failed validation.  The error message below will tell you where in your XML file the error occurred.  Here's how to fix it:

1) If the validation fails on something obvious, like an illegal attribute due to a spelling error (perhaps you used scorefnction instead of scorefunction), then you need to fix your XML file.
2) If you haven't run the XML rewriter script and this might be pre-2017 Rosetta XML, run the rewriter script (tools/xsd_xrw/rewrite_rosetta_script.py) on your input XML first.  The attribute values not being in quotes (scorefunction=talaris2014 instead of scorefunction="talaris2014") is a good indicator that this is your proble

In [5]:
d = dict(tppose.pose.scores)
from crispy_shifty.protocols.design import beta_nov16_terms

d = {k: v for k, v in d.items() if k not in beta_nov16_terms}

In [6]:
d

{'bb_clash': '250.36671447753906',
 'best_average_DAN_plddts': '0.9204101562',
 'best_average_plddts': '94.379172804',
 'best_model': '2',
 'best_ptm': '0.8931488654',
 'best_rmsd_to_input': '1.0943584816',
 'buns_parent': '2.0',
 'cms_AcB': '297.3140563964844',
 'cms_AnAc': '226.63072204589844',
 'cms_AnAcB': '640.595947265625',
 'cms_AnB': '347.1438293457031',
 'designed_by': 'mpnn',
 'docked_helix': '5',
 'dssp': 'LHHHHHHHHHHHHHHHHHHHHHHLLHHHHHHHHHHHHHHHLLLHHHHHHHHHHHHHHHHHHHHHHHLLHHHHHHHHHHHHHHHLLLLLHHHHHHHHHHHHHHHHHHHHHHLLHHHHHHHHHHHHHHHLLLHHHHHHHHHHHHHHHHHHHHHHHLLHHHHHHHHHHHHHHHLLHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHL',
 'elapsed_time': '5.834771156311035',
 'exposed_hydrophobics_parent': '7.770049571990967',
 'geometry_parent': '1.0',
 'holes_all_parent': '-1.5947035551071167',
 'holes_core_parent': '-2.432673215866089',
 'loop_dist': '11.541058243150694',
 'loop_sc': '0.7584340187061882',
 'looped_length': '171.0',
 'mean_pae': '4.042686462402344',
 'mean_pae_interaction': '4.7735304